In [1]:
import altair as alt
from itertools import chain
import requests
from nesta_ds_utils.loading_saving import S3 as nesta_s3
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from typing import NoReturn, List, Any
from time import time
import sentence_transformers
from sentence_transformers import SentenceTransformer

from discovery_child_development import S3_BUCKET, config, logging
from discovery_child_development.utils import openalex_utils
from discovery_child_development.utils import cluster_analysis_utils as cau

API_ROOT = config["openalex_keywords_api_root"]
S3_PATH = "metaflow/openalex_keyword_search"
YEARS = config["openalex_years"]
KEYWORDS = config["openalex_keywords"]
SEED = config["seed"]

load_dotenv()

model = SentenceTransformer("all-MiniLM-L6-v2")

alt.data_transformers.disable_max_rows()

/Users/rosie.oxbury/miniconda3/envs/discovery_child_development/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2023-11-29 15:02:36,982 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2023-11-29 15:02:37,612 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device: cpu


DataTransformerRegistry.enable('default')

In [2]:
queries = openalex_utils.generate_keyword_queries(API_ROOT, KEYWORDS, YEARS)

queries

["https://api.openalex.org/works?search=(abstract:(child OR infant OR baby OR prenatal OR pregnancy) AND abstract:('artificial intelligence' OR assess OR assessment OR 'augmented reality' OR autism OR behaviour OR development OR 'eye tracking' OR genetics OR income OR learning OR 'learning environment' OR monitor OR psychotherapy OR 'randomised controlled trials' OR robotics OR 'social media' OR 'social services' OR 'special need' OR technology OR 'virtual reality' OR wearable)) OR (title:(child OR infant OR baby OR prenatal OR pregnancy) AND title:('artificial intelligence' OR assess OR assessment OR 'augmented reality' OR autism OR behaviour OR development OR 'eye tracking' OR genetics OR income OR learning OR 'learning environment' OR monitor OR psychotherapy OR 'randomised controlled trials' OR robotics OR 'social media' OR 'social services' OR 'special need' OR technology OR 'virtual reality' OR wearable))&filter=publication_year:2019",
 "https://api.openalex.org/works?search=(abs

In [3]:
result = requests.get(queries[0])
# check that a single query runs ok
result

<Response [200]>

In [4]:
# find out how many hits we should get for all of the queries
total = 0

for query in queries:
    count = requests.get(query).json()["meta"]["count"]
    logging.info(f"Number of hits: {count}")
    total += count
    
total

2023-11-29 15:02:39,055 - root - INFO - Number of hits: 5319
2023-11-29 15:02:39,791 - root - INFO - Number of hits: 3062
2023-11-29 15:02:40,531 - root - INFO - Number of hits: 4714
2023-11-29 15:02:41,274 - root - INFO - Number of hits: 4754
2023-11-29 15:02:41,966 - root - INFO - Number of hits: 3660


21509

The metaflow script `pipeline/openalex/openalex_keyword_search.py` runs all of the queries and stores the results on S3. Below, we load the results and do a little bit of EDA.

In [5]:
INPUT_FILES = [
    f"openalex_keywords_True_year-{year}.json"
    for year in YEARS
]

In [6]:
openalex_df = openalex_utils.concat_json_files(INPUT_FILES, S3_BUCKET, S3_PATH)

len(openalex_df)

2023-11-29 15:02:41,993 - botocore.credentials - INFO - Found credentials in environment variables.
2023-11-29 15:02:51,339 - root - INFO - Number of works in openalex_keywords_True_year-2019.json: 5305
2023-11-29 15:02:56,618 - root - INFO - Number of works in openalex_keywords_True_year-2020.json: 3057
2023-11-29 15:03:04,212 - root - INFO - Number of works in openalex_keywords_True_year-2021.json: 4709
2023-11-29 15:03:11,703 - root - INFO - Number of works in openalex_keywords_True_year-2022.json: 4745
2023-11-29 15:03:17,839 - root - INFO - Number of works in openalex_keywords_True_year-2023.json: 3653


21469

In [7]:
# Retain only works in English
openalex_en = openalex_df[openalex_df["language"] == "en"]
openalex_en = openalex_en[openalex_en["abstract_inverted_index"].notnull()]
openalex_en = openalex_en[openalex_en["title"].notnull()]

In [8]:
openalex_en_abstracts = openalex_utils.create_text_data(
        openalex_en[["id", "title", "abstract_inverted_index"]]
    )

openalex_en_abstracts.head()

/Users/rosie.oxbury/Documents/git_repos/discovery_child_development/discovery_child_development/utils/openalex_utils.py:177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "abstract"] = df["abstract_inverted_index"].apply(
/Users/rosie.oxbury/Documents/git_repos/discovery_child_development/discovery_child_development/utils/openalex_utils.py:181: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "text"] = df["title"] + ". " + df["abstract"]


,id,title,abstract,text
0,https://openalex.org/W2992210402,An Artificial Somatic Reflex Arc,"Abstract The emulation of human sensation, per...",An Artificial Somatic Reflex Arc. Abstract The...
1,https://openalex.org/W2899856450,"Siri, Siri, in my hand: Who’s the fairest in t...",Artificial intelligence (AI)—defined as a syst...,"Siri, Siri, in my hand: Who’s the fairest in t..."
2,https://openalex.org/W2886354130,A systematic review of the smart home literatu...,A smart home is a residence equipped with smar...,A systematic review of the smart home literatu...
3,https://openalex.org/W2788388592,Continual lifelong learning with neural networ...,Humans and animals have the ability to continu...,Continual lifelong learning with neural networ...
4,https://openalex.org/W2923238705,The urgent need for microbiology literacy in s...,"Microbes and their activities have pervasive, ...",The urgent need for microbiology literacy in s...


In [9]:
openalex_docs = openalex_en_abstracts["text"].tolist()

In [10]:
# check out a random example
openalex_docs[10]

'Wearables and mobile technologies in Autism Spectrum Disorder interventions: A systematic literature review. Nowadays, in the Internet of Things era, wearables, mobile technologies and enhanced communication and computing capabilities has led to the upsurge of innovative mobile health solutions. Many research efforts have taken place recently in the domain of autism spectrum disorders (ASD). The current paper presents a thorough review of the literature on the use of wearables and mobile technologies for ASD-related interventions. It intends to give insights and guidelines to researchers in order to develop more useful and closer to market products. We searched seven databases for research articles published after 2000. Of 4,722 articles initially retrieved, only 83 papers met the inclusion criteria. Several challenges still exist in the research efforts towards the development of applications exploiting the latest wearables and mobile technologies for ASD interventions: small number 

In [11]:
# t0 = time()
# sentence_vectors_384 = model.encode(openalex_docs, show_progress_bar=True)
# print(f"vectorization done in {time() - t0:.3f} s")

In [12]:
# np.save("openalex_sentence_vectors_384.npy", sentence_vectors_384)

In [13]:
sentence_vectors_384 = np.load("openalex_sentence_vectors_384.npy")

In [14]:
umap_params = {
    "n_components": 50,  # apparently hdbscan does not work very well with more than 50 components
    "n_neighbors": 10,
    "min_dist": 0.5,
    "spread": 0.5,
}

In [15]:
# reduce dimensionality of the embeddings
sentence_vectors_50 = cau.umap_reducer(
    sentence_vectors_384, umap_params, random_umap_state=SEED
)

2023-11-29 15:03:18,477 - root - INFO - Generating 50-d UMAP embbedings for 16214 vectors


/Users/rosie.oxbury/miniconda3/envs/discovery_child_development/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [16]:
kmeans_labels = cau.kmeans_clustering(
    sentence_vectors_50, kmeans_params={'init': 'k-means++', 'n_clusters': 20}
)

2023-11-29 15:03:43,055 - root - INFO - Clustering 16214 vectors with K-Means clustering


/Users/rosie.oxbury/miniconda3/envs/discovery_child_development/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/rosie.oxbury/miniconda3/envs/discovery_child_development/lib/python3.10/site-packages/threadpoolctl.py:1010: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [17]:
# Reduce original vectors to 2D for plotting
openalex_texts_2d = cau.reduce_to_2D(sentence_vectors_384, random_state=SEED)

/Users/rosie.oxbury/miniconda3/envs/discovery_child_development/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [18]:
cluster_df = openalex_en_abstracts.assign(
    cluster=kmeans_labels,
    x=openalex_texts_2d[:, 0],
    y=openalex_texts_2d[:, 1],
)

In [19]:
# fig_hdbscan = (
#     alt.Chart(
#         cluster_df
#     )
#     .mark_circle()
#     .encode(
#         x="x",
#         y="y",
#         color=alt.Color("cluster:N", legend=alt.Legend(title="cluster")),
#         tooltip=["title", "cluster"],
#     )
#     .properties(width=800, height=600)
#     .interactive()
# )

# fig_hdbscan

In [20]:
CLUSTER_SUMMARY_MESSAGE = "Here are the most central texts of a cluster. \
Summarise what texts in this cluster are about in 2 sentences. \
\n\n##Abstracts\n\n {} \n\n##Description (2 short sentences)"

cluster_descriptions = cau.describe_clusters_with_gpt(
    cluster_df=cluster_df,
    embeddings=sentence_vectors_384,
    n_central=10,
    gpt_message=CLUSTER_SUMMARY_MESSAGE,
)

2023-11-29 15:03:54,916 - root - INFO - Cluster 0: ['Placebo From an Enactive Perspective. Due to their complexity and variability, placebo effects remain controversial. We suggest this is also due to a set of problematic assumptions (dualism, reductionism, individualism, passivity). We critically assess current explanations and empirical evidence and propose an alternative theoretical framework—the enactive approach to life and mind—based on recent developments in embodied cognitive science. We review core enactive concepts such as autonomy, agency, and sense-making. Following these ideas, we propose a move from binary distinctions (e.g., conscious vs. non-conscious) to the more workable categories of reflective and pre-reflective activity. We introduce an ontology of individuation, following the work of Gilbert Simondon, that allow us to see placebo interventions not as originating causal chains, but as modulators and triggers in the regulation of tensions between ongoing embodied an

In [21]:
cluster_names_dict = cau.generate_cluster_names_with_gpt(
    cluster_descriptions=cluster_descriptions,
)

2023-11-29 15:05:07,180 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [22]:
cluster_descriptions

['be seen in various domains such as sports, medicine, and technology. The ethical implications of Human Enhancement are discussed, with an emphasis on the need to focus on the practical effectiveness and impact of these enhancements in the real world.',
 'The first text is about the concept of Human Enhancement, which refers to the use of technology to improve the capabilities of the human body or create new ones. The author argues that Human Enhancement is a historical and pervasive phenomenon that has the potential to change the notion of being human. The second text discusses the concept of responsibility in the context of artificial intelligence (AI). The author proposes a broader framework of responsibility that includes moral responsibility, causal responsibility, relational responsibility, and narrative responsibility. They argue that humans have the responsibility to make sense of AI and its impact on society.',
 'derived from affective neuroscience, is that techne, including 

In [23]:
cluster_names_dict

{0: '1. Human Enhancement: Historical and Transformative',
 1: '2. Responsibility and AI: Broad Framework',
 2: '3. AI and Human Nature: Challenging Notions',
 3: '4. Human Enhancement: Ethical Implications',
 4: '5. Human Enhancement: Changing Relationship',
 5: '6. Human Enhancement: Natural Process',
 6: '7. Human Enhancement: Prosthetics and Modifications',
 7: '8. Human Enhancement: Dignity and Liberty',
 8: '9. AI and Semiotics: Understanding Cognition',
 9: '10. Human Enhancement: Practical Impact',
 10: '11. Technology and Emotions: Shaping Experience',
 11: '12. Placebo Effects: Alternative Framework',
 12: '13. Human Enhancement: Engaging with the World',
 13: '14. Responsibility and AI: Making Sense',
 14: '15. Placebo Effects: Reflective and Pre-Reflective Activity',
 15: '16. Human Enhancement: Adapting to Technology',
 16: '17. Responsibility and AI: Hermeneutic Responsibility',
 17: '18. Placebo Effects: Enactive Approach',
 18: '19. Human Enhancement: Historical and Wid